# Gemini-based Habermas Machine via AI Studio

The purpose of this notebook is to demonstrate the workings of the Habermas Machine, using publicly accessible models rather than the custom fine-tuned model. The Habermas Machine was introduced in this paper:

Tessler, M. H., Bakker, M. A., Jarrett D., Sheahan, H., Chadwick, M. J., Koster, R., Evans, G., Campbell-Gillingham, J., Collins, T., Parkes, D. C., Botvinick, M., and Summerfield, C. "AI can help humans find common ground in democratic deliberation." Science. (2024).

All parameters can be adjusted but as default this example uses:

*   Gemini 1.5 Flash via AIStudio
*   Chain-of-thought statement and ranking model
*   5 participants and 4 candidate statements
*   Example data from Table S50 (p. 265) in the Supplementary Materials.
*   The Habermas Machine is set to verbose mode.

In this example, we overwrite the winner from the opinion round with the winner in the actual experiment to demonstrate how the critiquing works from our real example. However, when you use your own opinions and critiques, this can be turned off.

# Setup and Importing Packages

In [ ]:
# Install dependencies from github.
# !pip install --upgrade git+https://github.com/google-deepmind/habermas_machine.git

import os

from habermas_machine import machine
from habermas_machine import types
from habermas_machine.social_choice import utils as sc_utils
# If error, check that you have pip installed `habermas_machine`.

# Configuration

In [ ]:
# Get API keys from https://aistudio.google.com/app/apikey.
os.environ['GOOGLE_API_KEY'] = ''

In [ ]:
QUESTION = 'Should the government provide universal free childcare from birth?'
NUM_CITIZENS = 5
NUM_CANDIDATES = 4
MODEL = 'gemini-1.5-flash'

statement_client = types.LLMCLient.AISTUDIO.get_client(MODEL)
reward_client = types.LLMCLient.AISTUDIO.get_client(MODEL)

statement_model = types.StatementModel.CHAIN_OF_THOUGHT.get_model()
reward_model = types.RewardModel.CHAIN_OF_THOUGHT_RANKING.get_model()
social_choice_method = types.RankAggregation.SCHULZE.get_method(
    tie_breaking_method=sc_utils.TieBreakingMethod.TBRC
)

# Initialize the Habermas Machine

In [ ]:
hm = machine.HabermasMachine(
        question=QUESTION,
        statement_client=statement_client,
        reward_client=reward_client,
        statement_model=statement_model,
        reward_model=reward_model,
        social_choice_method=social_choice_method,
        num_candidates=NUM_CANDIDATES,
        num_citizens=NUM_CITIZENS,
        verbose=True,
        num_retries_on_error=5,
)

# Opinion round

In [ ]:
OPINIONS = [
    (
        "The government should provide universal free childcare, but not"
        " necessarily from birth. High quality childcare allows parents to"
        " work, making use of their skills and experience and contributing to"
        " the economy. It also provides children with a positive environment in"
        " which to learn and to develop social skills. However, I do not"
        " necessarily support providing this childcare from birth. Babies"
        " benefit from having a consistent primary caregiver in their early"
        " months, and parents benefit from the opportunity to bond with their"
        " babies and to recover from the physical changes caused by birth and"
        " from the social changes caused by having a new family member. For"
        " this reason, I would support the government providing universal paid"
        " parental leave from birth, and providing universal free childcare"
        " from, say, 6 months old. I would also offer parents the opportunity"
        " to either use free childcare between 6 months and 1 year, or to have"
        " paid parental leave for the same period."
    ),
    (
        "I do not think that there should be universal free childcare,  I think"
        " free childcare places should be targeted at the people who cannot"
        " afford to pay for childcare, to encourage them back into the"
        " workplace. I think that some free childcare should be available to"
        " all children once they reach three, as long as it is in a setting"
        " which provides education as well as childcare."
    ),
    (
        "Given that most parents both need to work, in most instances free"
        " childcare would be of a huge advantage to younger demographic groups."
        " Although a number of the gender barriers have been broken down there"
        " are still differences in employment between men and women. On average"
        " men earn more than women. As a consequence it is more difficult to a"
        " man to be the main carer for children. It is also more stigmatised"
        " for men to be the main carers. Sort out these issues and there would"
        " be less need for child care. until then the cost of childcare can be"
        " disproportionate to the amount a parent is able to earn. In most"
        " cases drop off and pick up are the same times as the working day and"
        " mean that they are shortened. Whoever picks up nd drops off need"
        " special hours from their employer which can mean limitations to their"
        " career growth."
    ),
    (
        "Parents often struggle in the years between maternity leave finishing"
        " and the start of free childcare, especially in high cost of living"
        " areas. Often, one parent (usually the mother) has to give up work or"
        " reduce hours in order to provide childcare. This has a long term"
        " impact on the family as it impacts the caregiver's future earning"
        " potential as well as the family's immediate income. Often families"
        " feel forced into this situation because childcare costs are so high,"
        " and may not even cover all of the hours that the parents work so they"
        " have to pay extra to cover those additional hours. It would be good"
        " to offer stay at home parents the option of free childcare from birth"
        " or a basic income if they would prefer to stay at home with the"
        " child"
    ),
    (
        "Childcare is a very important aspect of a healthy society. In many"
        " parts of the world the extended family helps with childcare but in"
        " the UK this is not always possible and parents can be put under"
        " enormous pressure to look after young children and contribute to"
        " wider society. Having support in place for families with children is"
        " good for the parents and also the communities they are part of. The"
        " current state of the economy and the cost of living means that often"
        " both parents may have to work but when that is not the case, having"
        " downtime from raising a child is good for parental mental health."
    ),
]

In [ ]:
winner, sorted_statements = hm.mediate(OPINIONS)



Opinion round.

Question: Should the government provide universal free childcare from birth?

Opinions:
	Citizen 1: The government should provide universal free childcare, but not necessarily from birth. High quality childcare allows parents to work, making use of their skills and experience and contributing to the economy. It also provides children with a positive environment in which to learn and to develop social skills. However, I do not necessarily support providing this childcare from birth. Babies benefit from having a consistent primary caregiver in their early months, and parents benefit from the opportunity to bond with their babies and to recover from the physical changes caused by birth and from the social changes caused by having a new family member. For this reason, I would support the government providing universal paid parental leave from birth, and providing universal free childcare from, say, 6 months old. I would also offer parents the opportunity to either use fre

In [ ]:
print('Opinion round winner:')
print(winner)

# Uncomment to see the sorted statements.
# print('Opinion round sorted statements')
# for statement in sorted_statements:
#   print(statement)

# Overwrite winner (set to False to turn off).

In [ ]:
OVERWRITE_WINNER = True
HARDCODED_WINNER = (
    "In general, free childcare is a good thing, but it is important to"
    " consider how it is provided and for which age groups. We feel that it is"
    " important to offer support to parents in the form of parental leave, and"
    " that this should be available to both parents. In addition, we feel that"
    " free childcare should be provided from a young age, and that it should be"
    " provided in a way that supports children's development and learning, and"
    " not just as a childminding service. However, we do not feel that free"
    " childcare should be provided from birth, as we feel that it is important"
    " for babies to have a consistent primary caregiver in their early months."
    " For this reason, we would support the government providing universal paid"
    " parental leave from birth, and providing universal free childcare from,"
    " say, 6 months old. We would also offer parents the opportunity to either"
    " use free childcare between 6 months and 1 year, or to have paid parental"
    " leave for the same period."
)
if OVERWRITE_WINNER:
  hm.overwrite_previous_winner(HARDCODED_WINNER)

# Critique round

In [ ]:
CRITIQUES = [
    (
        "I agree with the arguments. I would add that we support parents being"
        " able to access paid childcare from birth if they need it - just not"
        " providing universal free childcare from birth."
    ),
    (
        "I do not agree that free childcare should be available from an early"
        " age but I do agree that any childcare should enhance a child's"
        " learning and development.  The idea of additional paid paternity"
        " leave is a good one, but if the additional costs are not covered by"
        " the government the could be crippling for small businesses."
    ),
    "This is very good\nBut can we add something about being irrespective",
    (
        "It's important to consider what form the free childcare would take and"
        " how many hours would be provided for free, it has to be enough to"
        " make going back to work worthwhile for the parent. Likewise, the paid"
        " parental leave needs to be a sufficient enough sum that the family"
        " can have a decent quality of life"
    ),
    (
        "I agree that the care should not just be a childminding service and it"
        " should support children's development. Universal paid parental leave"
        " is an excellent idea. The option to exchange childcare for parental"
        " leave is also a great idea."
    ),
]

In [ ]:
winner, sorted_statements = hm.mediate(CRITIQUES)

In [ ]:
print('Critique round winner:')
print(winner)

# Uncomment to see the sorted statements.
# print('Critique round sorted statements')
# for statement in sorted_statements:
#   print(statement)
